<h1>Encoding analysis</h1>

In [1]:
# Imports
import os 
import numpy as np

import encoding_helpers as helpers

%autosave 5

Autosaving every 5 seconds


<h2>Load features and data</h2>

In [5]:
# Modify this cell to change subject group. Currently set to 8-12 years
subject_group_str = '5-yrs'
print('Subject group:', subject_group_str)

Subject group: 5-yrs


In [6]:
# Data path
directory = os.path.join(os.path.expanduser('~'), 'Documents', 'Data')
print('Data path:', directory)

Data path: /Users/angira/Documents/Data


In [7]:
# Load BOLD data
bold_data = np.load(os.path.join(directory, subject_group_str, (subject_group_str+'_normalized_data.npy')))
print('Subject data:', np.shape(bold_data))

Subject data: (34, 183213, 152)


In [8]:
# Load features
social_features = np.load(os.path.join(directory, 'features', 'social_normalized.npy'))
print('Social features:', np.shape(social_features))

moten_features = np.load(os.path.join(directory, 'features', 'moten_reduced19.npy'))
print('Motion energy features:', np.shape(moten_features))

Social features: (152, 5)
Motion energy features: (152, 19)


In [9]:
# Combine features for use in model predictions

combined_features = np.concatenate((social_features, moten_features), axis=1)
print('Combined features:', np.shape(combined_features))

Combined features: (152, 24)


<h2>Generate test-train split</h2>

In [33]:
# ## test-train split with random states
# from sklearn import model_selection
# def get_continuous_split(data, start_index, end_index):
#     """
#         Generates a test / train split using continuous segments of the movie. 
#         Takes in data of the shape [TRS x samples]
#         Data should be divided using 0:30, 30:60, 60:90, 90:120, or 120: as test set.
#         Returns train, test sets
#     """
#     # Set testing set to match the indexes specified
#     test = data[start_index:end_index]
    
#     # Concatenate remaining data into training data
#     train = np.concatenate((data[:start_index], data[end_index:]), axis=0)
    
#     return train, test


In [10]:
subject_data = np.swapaxes(bold_data[0], 0, 1)

In [ ]:
# Random test / train split
from sklearn import model_selection

bold_train, bold_test, social_train, social_test, moten_train, moten_test = model_selection.train_test_split(
subject_data, social_features, moten_features,test_size=0.2, random_state=4)

combined_train, combined_test = model_selection.train_test_split(combined_features, test_size=0.2, random_state=4)

In [12]:
# Continuous test / train split
social_train, social_test = helpers.get_continuous_split(social_features, start_index=120, end_index=152)
moten_train, moten_test = helpers.get_continuous_split(moten_features, start_index=120, end_index=152)
bold_train, bold_test = helpers.get_continuous_split(subject_data, start_index=120, end_index=152)
combined_train, combined_test = helpers.get_continuous_split(combined_features, start_index=120, end_index=152)

<h2>Get encoding weights</h2>

In [55]:
subject_weights = helpers.get_banded_weights([social_features, moten_features], 
                                             [social_train, moten_train],
                                             [social_test, moten_test],
                                             bold_train, bold_test)

print(np.shape(subject_weights))

(24, 147260)


In [56]:
subject_predictions = helpers.get_predictions_from_weights(combined_test, subject_weights)
print(np.shape(subject_predictions))

(32, 147260)


In [57]:
subject_prediction_scores = helpers.get_prediction_scores(subject_predictions, bold_test)
print(np.shape(subject_prediction_scores))

(147260,)
